# Find a Shortest Transformation of One Genome into Another by 2-Breaks

[ba6d](https://rosalind.info/problems/ba6d/)

## 2-Break Sorting Problem

Find a shortest transformation of one genome into another by 2-breaks.

    Given: 

Two genomes with circular chromosomes on the same set of synteny blocks.

    Return: 

The sequence of genomes resulting from applying a shortest sequence of 2-breaks transforming one genome into the other.

In [185]:
import numpy as np

In [186]:
def chromosome_to_cycle(chrmosome):
    nodes = []
    
    for i in chrmosome:
        if i > 0:
            nodes.append(2 * i - 1)
            nodes.append(2 * i)
        else:
            nodes.append(-2 * i)
            nodes.append(-2 * i - 1)
    return nodes

In [187]:
def colored_edges(genome):
    edges = []
    for chromosome in genome:
        nodes = chromosome_to_cycle(chromosome)
        for j in range(len(chromosome)):
            edges.append((nodes[2 * j + 1], nodes[(2 * j + 2) % len(nodes)]))
    return edges

In [188]:
def colored_edges_cycles(blue, red):
	size = len(blue)+len(red)+1
	adj = np.zeros(shape=(size,2),dtype=int)
	visited = [0]*(size)
	for e in blue:
		adj[e[0],0] = e[1]
		adj[e[1],0] = e[0]
	for e in red:
		adj[e[0],1] = e[1]
		adj[e[1],1] = e[0]
	cycles = []
	for v in range(1,size):
		if visited[v]==1:
			continue
		visited[v]==1
		head = v
		c = [head]
		color = 0
		while(True):
			v = adj[v,color]
			if v == head:
				cycles.append(c)
				break
			visited[v] = 1
			c.append(v)
			color = (color+1)%2
	return cycles

In [189]:
def count_block(P,Q):
	s = set()
	for i in P[0]:
		s.add(abs(i))
	for i in Q[0]:
		s.add(abs(i))
	return len(s)

In [190]:
def two_break_distance(P,Q):
	blue = colored_edges(P)
	red = colored_edges(Q)
	cycles = colored_edges_cycles(blue,red)
	cycle_cnt = len(cycles)
	blocks_cnt = count_block(P,Q)
	return blocks_cnt - cycle_cnt

In [191]:
def graph_to_genome(graph):
	visited = []
	adj = [0 for x in range(len(graph) * 2 + 1)]
	for e in graph:
		adj[e[0]] = e[1]
		adj[e[1]] = e[0]
	chromosomes = []
	for e in graph:
		orig = e[0]
		if orig in visited:
			continue
		visited.append(orig)
		if orig%2 == 0:
			close = orig - 1
		else:
			close = orig + 1
		tmp = []
		while(True):
			if orig%2 == 0:
				tmp.append(int(orig/2))
			else:
				tmp.append(int(-(orig + 1)/2))
			dest = adj[orig]
			visited.append(dest)
			if dest == close:
				chromosomes.append(tmp)
				break
			
			if dest%2 == 0:
				orig = dest - 1
			else:
				orig = dest + 1
			visited.append(orig)
	return chromosomes

In [192]:
def two_break_on_genome(P, i, i_, j, j_):
	graph = colored_edges(P)
	graph = two_break_on_genome_graph(graph, i, i_, j, j_)
	return graph_to_genome(graph)

In [193]:
def two_break_on_genome_graph(graph, i, i_, j, j_):
    graph = [x for x in graph if x not in [(i, i_), (j, j_), (i_, i), (j_, j)]]
    graph.append((i, j))
    graph.append((i_, j_))
    return graph

In [194]:
def two_break_sorting(P,Q):
	red = colored_edges(Q)
	path = [P]
	
	while(two_break_distance(P, Q) > 0):
		blue = colored_edges(P)
		cycles = colored_edges_cycles(blue,red)
		for c in cycles:
			if len(c) >= 4:
				P = two_break_on_genome(P, c[0], c[1], c[3], c[2])
				path.append(P)
				break
	return path

In [195]:
def genome_str_to_list(genome):
    return list(map(int, genome.split(' ')))

In [196]:
def list_to_chromosome(genome):
    return ''.join(['(' + ' '.join(['+' + str(int(part)) if part > 0 else str(int(part)) for part in chrm]) + ')' for chrm in genome])

In [197]:
file = "rosalind_ba6d.txt"
with open(file, 'r') as f:
    lines = f.read().splitlines()
P = [genome_str_to_list(lines[0][1:-1])]
Q = [genome_str_to_list(lines[1][1:-1])]

transformations = two_break_sorting(P, Q)
for transformation in transformations:
    print(list_to_chromosome(transformation))


(-1 -7 +6 +9 -2 +12 +3 +11 -10 +8 -5 -4)
(-7 +6 +9 -2 +12 +3 +11 -10 +8 -5 -4)(-1)
(+6 +9 -2 +12 +3 +11 -10 +8 -5 -4 -7 +1)
(+6 +9 -2 +12 +3 +11 -10 +7 +4 +5 -8 +1)
(+6 +9 -2 +8 -5 -4 -7 +10 -11 -3 -12 +1)
(+9 -2 +8 -5 -4 -7 +10 -11 -3 -12 -6 -1)
(-2 +8 -5 -4 -7 +10 -11 -3 -12 -6 -1 -9)
(-2 +8 -5 -4 -7 +10 +1 +6 +12 +3 +11 -9)
(-2 +8 -5 -4 -7 +10 +1 +6 +12 -3 +11 -9)
(-2 +8 +3 -12 -6 -1 -10 +7 +4 +5 +11 -9)
(-2 +8 -4 -7 +10 +1 +6 +12 -3 +5 +11 -9)
